(AAEVALEVEC)=

# 2.3 Algoritmos y aplicaciones de eigenvalores y eigenvectores de una matriz

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion -p 8888:8888 -d palmoreck/jupyterlab_optimizacion:2.1.4`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion:2.1.4` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion).

```

---

Nota generada a partir de [liga](https://www.dropbox.com/s/s4ch0ww1687pl76/3.2.2.Factorizaciones_matriciales_SVD_Cholesky_QR.pdf?dl=0).

Ver comentario sobre Schur en https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html

https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.schur.html#scipy.linalg.schur

In [124]:
from scipy.linalg import schur, eigvals

In [125]:
A = np.array([[0, 2, 2], [0, 1, 2], [1, 0, 1]])

In [126]:
T, Z = schur(A)

In [127]:
T

array([[ 2.65896708,  1.42440458, -1.92933439],
       [ 0.        , -0.32948354, -0.49063704],
       [ 0.        ,  1.31178921, -0.32948354]])

In [128]:
evalue, evector = np.linalg.eig(T)

In [129]:
with np.printoptions(precision=3, suppress=True):
    print(evector)

[[1.   +0.j    0.497-0.081j 0.497+0.081j]
 [0.   +0.j    0.   +0.451j 0.   -0.451j]
 [0.   +0.j    0.737+0.j    0.737-0.j   ]]


In [130]:
print(evalue)

[ 2.65896708+0.j         -0.32948354+0.80225456j -0.32948354-0.80225456j]


In [131]:
A2 = T[1:3, 1:3]

In [132]:
A2

array([[-0.32948354, -0.49063704],
       [ 1.31178921, -0.32948354]])

In [133]:
evalue, evector = np.linalg.eig(A2)

In [134]:
evalue

array([-0.32948354+0.80225456j, -0.32948354-0.80225456j])